In [ ]:
import EN_17037_Recipes as enr 
import pyvista as pv
import os
import pandas as pd
import numpy as np
import trimesh as tm
import itertools

cwd = os.getcwd()
src = os.path.dirname(cwd)

In [ ]:
# create the material dictionary
material_dict = {
    "material_plastic": enr.material_plastic,
    "material_glass": enr.material_glass
}

# load the surface information table
surface_df = pd.read_csv("cv_file_table.csv")
surface_df

In [ ]:
# loading all meshes and converting them into hb surfaces
hb_surfaces = []
for _, srf in surface_df.iterrows():
    filepath = os.path.join(src, "data", srf["file_path"])
    mesh = pv.read(filepath)
    hb_surf = enr.pv_mesh_to_hbsurface(mesh, srf["surface_type"], srf["surface_name"], material_dict[srf["material_name"]])
    hb_surfaces.extend(hb_surf)

In [ ]:
# homogenous discretization of sphere
icosphere = tm.creation.icosphere(subdivisions=4, radius=1.0)
v = icosphere.vertices

# setting the rays and points
rays = [tuple(v) for v in icosphere.vertices]
grid_path = os.path.join(src, "data", "point_grid.csv")
grid_array = np.genfromtxt(grid_path, delimiter=",")
pnts = [tuple(pnt) for pnt in grid_array]

# generate grid of test points
test_inds = [j for j in range(len(pnts)) for i in range(len(rays))]
test_pnts = [pnt for pnt in pnts for i in range(len(rays))]
test_rays = rays * len(pnts)

analysis_grid = enr.AnalysisGrid.from_points_and_vectors(test_pnts, test_rays, 'test_grid')

In [ ]:
# put the recipe together
rp = enr.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)

In [ ]:
# write simulation to folder
project_name = 'EN_17037_context_view'
batch_file = rp.write(
    target_folder='.', project_name=project_name)


In [ ]:
# run the simulation
rp.run(batch_file, debug=False)

In [ ]:
# load rtrace results
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', usecols=[3,4,5], header=None, names=['last_int_name', 'first_int_dist', 'last_int_dist'])

# load the rays
rays_path = os.path.join(project_name, 'gridbased', project_name + '.pts')
rays_df = pd.read_csv(rays_path, skiprows=0, sep=' ', header=None, names=['Sx','Sy','Sz','Rx', 'Ry', 'Rz'])

# concatenate data frames and group by point ids
grid_results = pd.concat([rays_df, rtrace_res], axis = 1)
gb = grid_results.groupby(test_inds)

# extract point 0 info
point_0_results = gb.get_group(0)
point_0_results

In [ ]:
window_mask = (point_0_results['last_int_dist'] - point_0_results['first_int_dist']).abs() > 0
print('window view solid angle:',4 * np.pi * window_mask.sum()/len(window_mask), 'steradian')

In [ ]:
horizontal_line_mask = point_0_results['Rz'].abs() < 0.01
print('horizontal window view angle:',(2 * np.pi * window_mask * horizontal_line_mask).sum() / horizontal_line_mask.sum(), 'radians')

In [ ]:
view_quality_ratio = point_0_results['last_int_name'].value_counts() / len(point_0_results)
view_quality_ratio

In [ ]:
p = pv.Plotter(notebook=True)

icosphere = tm.creation.icosphere(subdivisions=4, radius=1.0)

p.add_mesh(icosphere.vertices, scalars=point_0_results['last_int_dist'], point_size=10)

p.add_axes()

p.show(use_ipyvtk=True)